In [1]:
### starts from raw AMT ratings
import pandas as pd
import numpy as np
import csv
import os
%pprint off

Pretty printing has been turned OFF


In [2]:
path = '/Users/songkim/GoogleDrive/Primary/Projects/VerbVector/AMT/html/experiments/ratings2/results/'
backups = next(os.walk(path))[1]
backups = [b for b in backups if b.startswith('backup')]
backups

['backup0_1_550', 'backup_210813', 'backup1_550-1050', 'backup3_0628', 'backup2_0610']

In [3]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

In [4]:
def return_res_catch_subj(backupfolder):
    
    path = '/Users/songkim/GoogleDrive/Primary/Projects/VerbVector/AMT/html/experiments/ratings2/results/'
#     stimr = pd.read_csv(path+backupfolder+'/'+'wro1_stimuli.txt', delimiter="\t")
#     stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
#     stim['stim'] = stim['stim'].str[:-1]

    catchr = pd.read_csv(path+backupfolder+'/'+'wro1_catchresults.txt',delimiter="\t")
    catch = catchr.rename(columns={"response": "catch_response", "subject_id":"catch_subject_id"})

    resr = pd.read_csv(path+backupfolder+'/'+'wro1_results.txt', delimiter="\t")
    res = resr.drop(columns=['stimtype_id','sense'])

    subj_raw = pd.read_csv(path+backupfolder+'/'+'wro1_subjects.txt',delimiter="\t")
    subj = subj_raw.rename(columns={"id": "subject_id"})

    return (res, catch, subj)

In [5]:
#### basic check on each dataset ##### 
b1= backups[0]
b2= backups[1]

for b in [b1, b2]:
    print('backup selected:', b)
    print(os.listdir(path+'/'+b))

    stimr = pd.read_csv(path+b+'/'+'wro1_stimuli.txt', delimiter="\t") # r means 'raw'
    stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
    stim['stim'] = stim['stim'].str[:-1]

    ques = pd.read_csv(path+b+'/'+'wro1_questions.txt', delimiter="\t")
    catchr = pd.read_csv(path+b+'/'+'wro1_catchresults.txt',delimiter="\t")
    catch = catchr.rename(columns={"response": "catch_response", "subject_id":"catch_subject_id"})
    print ("catch trial:", catch.shape[0])

    resr = pd.read_csv(path+b+'/'+'wro1_results.txt', delimiter="\t")
    res = resr.drop(columns=['stimtype_id','sense'])
    #print ("main trial:", res.shape[0])
    starttime = res.loc[res['id']==1, "start"].values[0]
    endtime = res.loc[res['id']==res.shape[0], "end"].values[0]
    print('This data contains ratings collected from', starttime, 'to', endtime)

    subj_raw = pd.read_csv(path+b+'/'+'wro1_subjects.txt',delimiter="\t")
    subj = subj_raw.rename(columns={"id": "subject_id"})
    print('subject N:', subj.shape[0])

    print('# of questions:', ques.shape[0])
    print('# of catch HITs:', catch.shape[0])
    print('# of HITs:     ', res.shape[0])
    print('# of subjects: ', subj.shape[0])

    check1 = catch.shape[0] == res.shape[0]
    check2 = res.shape[0] >= subj.shape[0] 
    check3 = res['subject_id'].equals(other=catch['catch_subject_id'])

    if check1==0:
        print ('# of catch trials != # of trials')
    elif check2==0:
        print ('# of subjects are more than # of HITs')
    elif check3==0:
        print('subject_id are not the same in trials and catchtrials')

    print ('=============================================') 


backup selected: backup0_1_550
['wro1_stimuli.txt', 'wro1_subjects.txt', 'wro1_catchresults.txt', 'wro1_stimtypes.txt', 'wro1_results.txt', 'wro1_questions.txt']
catch trial: 554
This data contains ratings collected from 2021-05-25 16:26:41 to 2021-05-28 01:18:52
subject N: 527
# of questions: 72
# of catch HITs: 554
# of HITs:      554
# of subjects:  527
backup selected: backup_210813
['0813_wro1_results.csv', '0813_wro1_questions.csv', '0813_wro1_subjects.csv', 'wro1_stimuli.txt', 'wro1_subjects.txt', 'wro1_catchresults.txt', '0813_wro1_catchresults.csv', 'wro1_results.txt', 'wro1_questions.txt', '0813_wro1_stimuli.csv']
catch trial: 3781
This data contains ratings collected from 2021-05-28 16:52:09 to 2021-08-13 10:28:09
subject N: 1294
# of questions: 72
# of catch HITs: 3781
# of HITs:      3781
# of subjects:  1294


In [6]:
######catch trial info
catch1 = 'being an animal living on the surface of sun'
catch2 = 'being something about which you are supposed to answer questions in this assignment'
catch3 = 'being an object that is smaller than a shoe box and larger than a mountain'
catch4 = 'being something you have thought about within the last couple minutes'
catch5 = 'being used to check whether you are carefully reading the questions'
catch_answers = ['0/7','6','0','6','6']

# def count_correct_answers(list1):
#     catch_answers = ['0/7','6','0','6','6']
#     if len(list1) != len(catch_answers):
#         print ('responses have different lengths!')
#         return
#     correct_answer = 0
#     if list1[0] in catch_answers[0].split('/'):
#         correct_answer += 1
#     for i in range(1,len(list1)):
#         iscorrect = list1[i]==catch_answers[i]
#         correct_answer += iscorrect        
#     return correct_answer

def count_incorrect_answers(list1):
    catch_answers = ['0/7','6','0','6','6']
    if len(list1) != len(catch_answers):
        print ('responses have different lengths!')
        return
    correct_answer = 0
    if list1[0] in catch_answers[0].split('/'):
        correct_answer += 1
    for i in range(1,len(list1)):
        iscorrect = list1[i]==catch_answers[i]
        correct_answer += iscorrect        
    return (5-correct_answer)

count_incorrect_answers(['7', '6', '0', '6', '6'])

0

In [7]:
def merge_tables(backupfolder): 
    '''function that merges results table, catch table, and subject table 
       backupfolder = the rating folder of interest'''
    [res, catch, subj] = return_res_catch_subj(backupfolder)
    
    res_all = pd.merge(left=res, right=subj, on='subject_id')
    res_all = res_all.sort_values(by=['id'])
    print('result df:', res.shape)
    print('subject df:', subj.shape)
    print('df returned:', res_all.shape)
    
    catch = catch.drop(columns='id') 
    res_all = pd.merge(left=res_all, right=catch, left_on="id", right_on="results_id")
    
    #print(res_all.shape)
    #print(catch.shape)
    print ('=========================================================')
    
    return res_all

In [8]:
# merge each data 
b1= backups[0]
b2= backups[1]
res_all1 = merge_tables(b1)
res_all2 = merge_tables(b2)

result df: (554, 9)
subject df: (527, 6)
df returned: (554, 14)
result df: (3781, 9)
subject df: (1294, 6)
df returned: (3781, 14)


In [9]:
res_combined = pd.concat([res_all1, res_all2], ignore_index=True)
res_combined['id'] = list(range(1, res_combined.shape[0]+1))

print(res_all1.shape)
print(res_all2.shape)
print(res_combined.shape)
res_combined

##check whether there is a duplicate row
duplicates = res_combined.duplicated()
duplicates = duplicates.index[duplicates]
len(duplicates)


#### add lemma column
stimr = pd.read_csv(path+b1+'/'+'wro1_stimuli.txt', delimiter="\t")
stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
stim['stim'] = stim['stim'].str[:-1]
stim.rename(columns={'id':'stim_id'}, inplace=True)
stim

res_combined = pd.merge(left=res_combined, right=stim, on="stim_id").sort_values(by="id")
res_combined = rearrange_col(res_combined, 'stim', 'stim_id')


##### mark bad subjects (updated)
res_combined['catch_response_split'] = res_combined['catch_response'].apply(lambda x:x.split('_'))
catch_series = res_combined['catch_response_split'].tolist()
#correct_answer = [count_correct_answers(l) for l in catch_series]
incorrect_answer = [count_incorrect_answers(l) for l in catch_series]
res_combined['incorrectN'] = incorrect_answer

res_combined.loc[res_combined['incorrectN']>=3, 'badsubject'] = 'Y'
res_combined.loc[res_combined['incorrectN']<3, 'badsubject'] = 'N'
res_bad = res_combined.loc[res_combined['badsubject']=='Y']
print('# of bad subject:', res_bad.shape[0], 'out of', res_combined.shape[0])

res_combined['response_split'] = res_combined['response'].apply(lambda x:x.split('_'))
res_combined['RT_split'] = res_combined['rt'].apply(lambda x:x.split('_'))
res_combined['buttonpress_split'] = res_combined['buttonpress'].apply(lambda x:x.split('_'))

res_combined = rearrange_col(res_combined, "response_split", "response")
res_combined = rearrange_col(res_combined, "RT_split", "rt")
res_combined = rearrange_col(res_combined, "buttonpress_split", "buttonpress")

for c in ['catch_response', 'response', 'rt', 'buttonpress']:
    res_combined.drop(columns=[c], inplace=True)
    
res_combined.rename(columns={'catch_response_split': 'catch_response', 'response_split':'response', 'RT_split':'RT', 'buttonpress_split':'buttonpress'}, inplace=True)

res_combined

(554, 17)
(3781, 17)
(4335, 17)
# of bad subject: 33 out of 4335


,id,stim_id,stim,subject_id,response,start,end,RT,buttonpress,turkcode,subject_num,gender,age,education,occupation,results_id,catch_subject_id,catch_response,incorrectN,badsubject
0,1,159,laugh,1,"[1, 0, 0, 3, 0, 0, 1, 5, 6, 2, 3, 3, 6, 2, 0, ...",2021-05-25 16:26:41,2021-05-25 16:35:15,"[10451, 2940, 2319, 3999, 2779, 3110, 6525, 75...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, ...",8545510,A3I9XLIHPPWPN1,f,57,18,Dataanalystadministrationassistant,1,1,"[0, 6, 0, 6, 6]",0,N
14,2,295,see,2,"[6, 3, 0, 1, 1, 2, 0, 4, 0, 1, 0, 3, 3, 0, 0, ...",2021-05-25 16:29:38,2021-05-25 16:36:24,"[4024, 9148, 2726, 5130, 5199, 3487, 2790, 432...","[1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2727964,A2CWJRAEFZ44HU,m,36,14,nursingassistantihelpoutpatientsandstaffinahos...,2,2,"[0, 6, 0, 6, 6]",0,N
26,3,214,flatten,3,"[3, 7, 7, 7, 5, 3, 1, 1, 3, 2, 0, 5, 7, 1, 4, ...",2021-05-25 16:26:58,2021-05-25 16:36:47,"[4432, 2912, 2592, 3640, 7976, 6904, 5368, 608...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3205581,A7ERZELTAMWL5,m,70,18,PatrolofficerIridearoundaresidentialcomplextoc...,3,3,"[0, 6, 0, 6, 6]",0,N
38,4,195,increase,4,"[4, 3, 6, 4, 3, 2, 3, 3, 6, 5, 5, 5, 6, 6, 5, ...",2021-05-25 16:30:25,2021-05-25 16:37:35,"[2941, 3771, 4392, 2726, 3770, 3690, 2473, 130...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",6434648,A1MKYMYY34DZIO,m,30,16,student,4,4,"[4, 6, 5, 6, 6]",2,N
52,5,262,calm,5,"[4, 0, 0, 0, 3, 0, 0, 1, 4, 0, 5, 0, 4, 2, 2, ...",2021-05-25 16:28:07,2021-05-25 16:38:09,"[4142, 2349, 7257, 8933, 4802, 3372, 3349, 830...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, ...",1247441,A2LF84L3K71GR2,f,38,12,RestaurantSupervisor,5,5,"[0, 6, 0, 6, 6]",0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4158,4331,103,shorten,1293,"[5, 5, 5, 5, 5, 6, 4, 6, 6, 5, 4, 3, 4, 4, 4, ...",2021-08-12 23:06:49,2021-08-12 23:40:48,"[667924, 14420, 20586, 4278, 2270, 6312, 9264,...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2285767,APG9EK6P41THO,m,48,16,NaN,3777,1293,"[0, 6, 0, 6, 6]",0,N
3803,4332,243,discover,1294,"[3, 5, 4, 5, 4, 5, 4, 4, 5, 4, 5, 5, 4, 5, 5, ...",2021-08-12 23:37:43,2021-08-12 23:47:34,"[1992, 1873, 1567, 3326, 2023, 3199, 1328, 270...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1553234,A4SQSL2ZCEU1,f,48,3,15,3778,1294,"[4, 6, 0, 6, 6]",1,N
1341,4333,4,walk,851,"[0, 1, 0, 0, 0, 2, 0, 5, 6, 6, 4, 0, 0, 6, 0, ...",2021-08-13 00:42:37,2021-08-13 00:48:49,"[3105, 3712, 4184, 3791, 4005, 6582, 2013, 629...","[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, ...",9692119,AFO9MCOXXSYY7,f,43,16,customerservicerep,3779,851,"[0, 6, 0, 0, 6]",1,N
4235,4334,198,decrease,210,"[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 5, 0, 4, 1, ...",2021-08-13 03:32:09,2021-08-13 03:37:55,"[1976, 4112, 2681, 2408, 2120, 7560, 8361, 344...","[1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, ...",8451568,A2LV5432PV1S39,m,34,16,DeputyCourtClerk,3780,210,"[0, 6, 0, 6, 6]",0,N


In [10]:
## export to csv
#res_combined.to_csv(path+'results_clean_0813.csv', index=False)
res_combined.to_csv(path+'results_catchreject_0813.csv', index=False)

In [11]:
#######to export bad subjects list
df_bad = res_combined.loc[res_combined['badsubject']=='Y', ]
df_bad.to_csv(path+'badsubjects_0813.csv')